In [1]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm
import json

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df['Date']=pd.to_datetime(df['Date'])
    df['TestGroup'] = (df['Employee'].isin((
        '364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275'
        ))).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    # for queue in df['TaskQueue'].unique():
    #     df[queue] = (df['TaskQueue']==queue).astype(int)
    df['Constant'] =1
    return df

# Run query
Uses a fuzzy join to see how many helix searchs/expert assist messages an expert had durring the time where they were taking a call.

In [4]:
dfp = run_query(file_to_string('AHT And Sales - With Helix and Expert Assist Fuzzy Join.SQL'))
display(dfp.describe(),dfp.head())

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_13056\3077383477.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,HandleTime,Offered,Accepted,HelixSearches,ExpertAssistSearches
count,"70,847.00","70,847.00","70,847.00","70,847.00","70,847.00","70,847.00"
mean,"560,924.49","1,127.20",0.28,0.03,0.05,0.01
std,"33,911.04",966.88,0.45,0.18,0.35,0.26
mad (mean),"18,001.72",694.26,0.40,0.06,0.10,0.03
min,"293,932.00",0.00,0.00,0.00,0.00,0.00
25%,"550,446.00",478.00,0.00,0.00,0.00,0.00
50%,"572,247.00",867.00,0.00,0.00,0.00,0.00
75%,"575,731.00","1,481.00",1.00,0.00,0.00,0.00
max,"581,694.00","13,139.00",1.00,1.00,15.00,19.00


,Date,Employee,HandleTime,Offered,Accepted,HelixSearches,ExpertAssistSearches
0,2023-06-23,573276,988,0,0,0,0
1,2023-08-14,573276,511,1,0,0,0
2,2023-06-07,573276,1510,0,0,0,0
3,2023-05-18,573276,531,0,0,0,0
4,2023-07-18,573276,2455,1,0,0,0


In [ ]:
df =dfp.copy()
df = add_conditionals(df)
pivot = pd.pivot_table(df,index = 'Date',columns = 'TestGroup',values='HelixSearches',aggfunc=[sum,len])
display(pivot)
pivot['Helix Usage']=pivot['sum']/pivot['len']
px.line(pivot['Helix Usage'])

# Graphical Analysis And Summary Stats of Sessions Using Each combo of tools

In [5]:
df = dfp.copy()

conds = [np.logical_and(df['HelixSearches']>=1,df['ExpertAssistSearches']>=1),df['ExpertAssistSearches']>=1,df['HelixSearches']>=1]
choices=['Both','Expert Assist','Helix']
df['Tool Used'] = np.select(condlist=conds,choicelist=choices,default='None')
display(df.head())
# px.strip(df,x='Tool Used',y='HandleTime').show()
# px.box(df[df['Tool Used'].isin(['Helix','Expert Assist'])],x='Tool Used',y='HandleTime').show()
# px.strip(df[df['Tool Used'].isin(['Helix','Expert Assist'])],x='Tool Used',y='HandleTime').show()
# px.box(df,x='Tool Used',y='HandleTime').show()
percentile_List = [.5,.75,.9,.95] #[.1,.2,.3,.4,.5,.6,.7,.8,.9]
# Build out a dataframe for the handle time summaries
describe_df = pd.DataFrame(df['HandleTime'].describe(percentiles=percentile_List)).rename({'HandleTime':'Total With or Without Tools'},axis=1)
for tool in ['None','Expert Assist','Helix','Both']:
        print(tool)
        describe_df=describe_df.join(pd.DataFrame(df[df['Tool Used']==tool]['HandleTime'].describe(percentiles=percentile_List)).rename({'HandleTime':tool},axis=1))
#         px.histogram(df[df['Tool Used']==tool],color='Tool Used',x='HandleTime',range_x=[0,10000],nbins=10,histnorm='percent').show()
        
        
display(describe_df)
del df

,Date,Employee,HandleTime,Offered,Accepted,HelixSearches,ExpertAssistSearches,Tool Used
0,2023-06-23,573276,988,0,0,0,0,None
1,2023-08-14,573276,511,1,0,0,0,None
2,2023-06-07,573276,1510,0,0,0,0,None
3,2023-05-18,573276,531,0,0,0,0,None
4,2023-07-18,573276,2455,1,0,0,0,None


None
Expert Assist
Helix
Both


,Total With or Without Tools,None,Expert Assist,Helix,Both
count,"70,847.00","68,105.00",389.00,"2,329.00",24.00
mean,"1,127.20","1,099.89","1,664.39","1,828.45","1,872.42"
std,966.88,939.18,"1,165.93","1,356.45","1,085.78"
mad (mean),694.26,674.88,873.02,"1,008.59",849.08
min,0.00,0.00,5.00,0.00,302.00
50%,867.00,849.00,"1,379.00","1,488.00","1,745.00"
75%,"1,481.00","1,446.00","2,135.00","2,421.00","2,386.50"
90%,"2,335.40","2,273.60","3,250.80","3,599.40","3,288.00"
95%,"3,009.00","2,934.00","4,111.20","4,456.00","3,549.40"
max,"13,139.00","13,139.00","7,053.00","10,355.00","4,559.00"
